**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Production economy and CO2 taxation](#toc1_)    
- 2. [Problem 2: Career choice model](#toc2_)    
- 3. [Problem 3: Barycentric interpolation](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
# Write your code here
import numpy as np
from types import SimpleNamespace

## 1. <a id='toc1_'></a>[Problem 1: Production economy and CO2 taxation](#toc0_)

Consider a production economy with two firms indexed by $j \in \{1,2\}$. Each produce its own good. They solve

$$
\begin{align*}
\max_{y_{j}}\pi_{j}&=p_{j}y_{j}-w_{j}\ell_{j}\\\text{s.t.}\;&y_{j}=A\ell_{j}^{\gamma}.
\end{align*}
$$

Optimal firm behavior is

$$
\begin{align*}
\ell_{j}^{\star}(w,p_{j})&=\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}} \\
y_{j}^{\star}(w,p_{j})&=A\left(\ell_{j}^{\star}(w,p_{j})\right)^{\gamma}
\end{align*}
$$

The implied profits are

$$
\pi_{j}^*(w,p_{j})=\frac{1-\gamma}{\gamma}w\cdot\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}}
$$

A single consumer supplies labor, and consumes the goods the firms produce. She also recieves the implied profits of the firm.<br>
She solves:

$$
\begin{align*}
U(p_1,p_2,w,\tau,T) = \max_{c_{1},c_{2},\ell} & \log(c_{1}^{\alpha}c_{2}^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} \\
\text{s.t.}\,\,\,&p_{1}c_{1}+(p_{2}+\tau)c_{2}=w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})
\end{align*}
$$

where $\tau$ is a tax and $T$ is lump-sum transfer. <br>
For a given $\ell$, it can be shown that optimal behavior is

$$
\begin{align*}
c_{1}(\ell)&=\alpha\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{1}} \\
c_{2}(\ell)&=(1-\alpha)\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{2}+\tau} \\
\end{align*}
$$
Such that optimal behavior is:
$$
\ell^* = \underset{\ell}{\arg\max} \log(\left(c_{1}(\ell)\right)^{\alpha}\cdot \left(c_{2}(\ell)\right)^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} 
$$
With optimal consumption:
$$
\begin{align*}
c_1^*=c_{1}(\ell^*) \\
c_2^*=c_{2}(\ell^*)\\
\end{align*}
$$


The government chooses $\tau$ and balances its budget so $T=\tau c_2^*$. We initially set $\tau,T=0$.

Market clearing requires:

1. Labor market: $\ell^* = \ell_1^* + \ell_2^*$
1. Good market 1: $c_1^* = y_1^*$
1. Good market 2: $c_2^* = y_2^*$


**Question 1:** Check market clearing conditions for $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`. We choose $w=1$ as numeraire.

In [ ]:
import numpy as np                          # Numerical library for arrays, grids, and math
from types import SimpleNamespace           # Lightweight container for parameters and solutions


# ======================================================
# Namespaces
# ======================================================

par = SimpleNamespace()                     # Container for all model parameters
sol = SimpleNamespace()                     # Container for solutions (used later)


# ======================================================
# Parameters
# ======================================================

# firms
par.A = 1.0                                 # Productivity parameter in production function
par.gamma = 0.5                             # Labor share parameter (0 < gamma < 1)

# households
par.alpha = 0.3                             # Preference weight on good 1
par.nu = 1.0                                # Disutility of labor scale parameter
par.epsilon = 2.0                           # Curvature of labor disutility

# government
par.tau = 0.0                               # Consumption tax on good 2
par.T = 0.0                                 # Lump-sum transfer

# Question 3 (not used here)
par.kappa = 0.1                             # Placeholder parameter for later extensions


# ======================================================
# Firm block
# ======================================================

def firm_labor_star(w, p, par):
    # Optimal labor demand from firm j given wage w and price p
    return (p * par.A * par.gamma / w) ** (1.0 / (1.0 - par.gamma))


def firm_output_star(w, p, par):
    # Compute optimal labor demand
    ell = firm_labor_star(w, p, par)
    # Production function evaluated at optimal labor
    return par.A * ell ** par.gamma


def firm_profit_star(w, p, par):
    # Compute optimal labor demand
    ell = firm_labor_star(w, p, par)
    # Closed-form expression for profits at the optimum
    return ((1.0 - par.gamma) / par.gamma) * w * ell


# ======================================================
# Household block
# ======================================================

def consumer_income(ell, w, par, pi1, pi2):
    # Household income: labor income + transfers + firm profits
    return w * ell + par.T + pi1 + pi2


def c1_of_ell(ell, w, p1, par, pi1, pi2):
    # Compute total income
    I = consumer_income(ell, w, par, pi1, pi2)
    # Cobb–Douglas demand for good 1
    return par.alpha * I / p1


def c2_of_ell(ell, w, p2, par, pi1, pi2):
    # Compute total income
    I = consumer_income(ell, w, par, pi1, pi2)
    # Cobb–Douglas demand for good 2, including consumption tax
    return (1.0 - par.alpha) * I / (p2 + par.tau)


def household_objective(ell, w, p1, p2, par, pi1, pi2):

    # Penalize non-positive labor to keep optimizer in admissible region
    if ell <= 0.0:
        return -1e14                         # Large negative value since we maximize

    # Consumption choices implied by labor choice
    c1 = c1_of_ell(ell, w, p1, par, pi1, pi2)
    c2 = c2_of_ell(ell, w, p2, par, pi1, pi2)

    # Penalize infeasible consumption
    if c1 <= 0.0 or c2 <= 0.0:
        return -1e14

    # Utility from consumption (log Cobb–Douglas)
    util = np.log((c1 ** par.alpha) * (c2 ** (1.0 - par.alpha)))

    # Disutility from labor
    disutil = par.nu * ell ** (1.0 + par.epsilon) / (1.0 + par.epsilon)

    # Net utility
    return util - disutil


# ======================================================
# Simple optimizer (grid + refinement)
# ======================================================

def maximize_1d(f, lb, ub, n_grid=2000, n_refine=4):

    # Construct an evenly spaced grid on the full interval
    grid = np.linspace(lb, ub, n_grid)

    # Evaluate objective function on the grid
    vals = np.array([f(x) for x in grid])

    # Initial maximizer from coarse grid
    x_star = grid[np.argmax(vals)]

    # Initial search width
    width = (ub - lb) / n_grid

    # Local refinement loop
    for _ in range(n_refine):

        # Define local bounds around current maximizer
        lb_loc = max(lb, x_star - width)
        ub_loc = min(ub, x_star + width)

        # Construct refined local grid
        grid = np.linspace(lb_loc, ub_loc, n_grid)

        # Re-evaluate objective function locally
        vals = np.array([f(x) for x in grid])

        # Update maximizer
        x_star = grid[np.argmax(vals)]

        # Shrink search width
        width = (ub_loc - lb_loc) / n_grid

    # Return approximate maximizer
    return x_star


def solve_household(par, w, p1, p2, pi1, pi2, ell_upper=500.0):

    # Objective function as a function of labor only
    def obj(ell):
        return household_objective(ell, w, p1, p2, par, pi1, pi2)

    # Create solution container
    hh = SimpleNamespace()

    # Solve for optimal labor supply
    hh.ell = maximize_1d(obj, 0.0, ell_upper)

    # Compute implied consumption choices
    hh.c1 = c1_of_ell(hh.ell, w, p1, par, pi1, pi2)
    hh.c2 = c2_of_ell(hh.ell, w, p2, par, pi1, pi2)

    # Return household solution
    return hh


# ======================================================
# Market clearing
# ======================================================

def market_clearing(w, p1, p2, par, ell_upper=500.0):

    # Create solution container
    sol = SimpleNamespace()

    # firms: good 1
    sol.ell1 = firm_labor_star(w, p1, par)     # Labor demand firm 1
    sol.y1 = firm_output_star(w, p1, par)      # Output firm 1
    sol.pi1 = firm_profit_star(w, p1, par)     # Profits firm 1

    # firms: good 2
    sol.ell2 = firm_labor_star(w, p2, par)     # Labor demand firm 2
    sol.y2 = firm_output_star(w, p2, par)      # Output firm 2
    sol.pi2 = firm_profit_star(w, p2, par)     # Profits firm 2

    # household
    sol.hh = solve_household(
        par=par,
        w=w,
        p1=p1,
        p2=p2,
        pi1=sol.pi1,
        pi2=sol.pi2,
        ell_upper=ell_upper
    )

    # market-clearing residuals
    sol.res_labor = sol.hh.ell - (sol.ell1 + sol.ell2)  # Labor market
    sol.res_good1 = sol.hh.c1 - sol.y1                  # Good 1 market
    sol.res_good2 = sol.hh.c2 - sol.y2                  # Good 2 market

    # Return solution object
    return sol


# ======================================================
# Equilibrium error
# ======================================================

def equilibrium_error(w, p1, p2, par):
    # Compute market-clearing residuals
    sol = market_clearing(w, p1, p2, par)
    # Sum of squared residuals (scalar equilibrium criterion)
    return (
        sol.res_labor**2
        + sol.res_good1**2
        + sol.res_good2**2
    )


# ======================================================
# Question 1: grid search over prices
# ======================================================

if __name__ == "__main__":

    w = 1.0                                  # Wage normalized to one (numeraire)

    p1_grid = np.linspace(0.1, 2.0, 100)    # Grid for price of good 1
    p2_grid = np.linspace(0.1, 2.0, 100)    # Grid for price of good 2

    # Storage for equilibrium error
    E = np.empty((len(p1_grid), len(p2_grid)))

    # Loop over price pairs
    for i, p1 in enumerate(p1_grid):
        for j, p2 in enumerate(p2_grid):
            E[i, j] = equilibrium_error(w, p1, p2, par)

    # Index of minimum equilibrium error
    idx = np.unravel_index(np.argmin(E), E.shape)

    # Equilibrium prices
    p1_ast = p1_grid[idx[0]]
    p2_ast = p2_grid[idx[1]]

    # Print equilibrium prices
    print(f"p1* = {p1_ast:.2f}")
    print(f"p2* = {p2_ast:.2f}")

    # Verify market clearing at equilibrium
    sol_ast = market_clearing(w, p1_ast, p2_ast, par)

    print("Labor residual:", sol_ast.res_labor)
    print("Good 1 residual:", sol_ast.res_good1)
    print("Good 2 residual:", sol_ast.res_good2)

p1* = 0.98
p2* = 1.48
Labor residual: 0.003920306740898827
Good 1 residual: -0.007670863007228912
Good 2 residual: 0.007733369720193339


**Question 2:** Find the equilibrium prices $p_1$ and $p_2$.<br>
*Hint: you can use Walras' law to only check 2 of the market clearings*

In [ ]:
def equilibrium_error_walras(w, p1, p2, par):
    """
    Equilibrium error using Walras' Law:
    Only the two goods markets are checked.
    If both goods markets clear, the labor market
    clears automatically.
    """
    sol = market_clearing(w, p1, p2, par)     # Compute market-clearing residuals
    return sol.res_good1**2 + sol.res_good2**2  # Sum of squared goods-market residuals


# =========================
# Question 2: equilibrium prices
# =========================

w = 1.0                                      # Wage normalized to one (numeraire)

p1_grid = np.linspace(0.1, 2.0, 200)         # Grid for price of good 1
p2_grid = np.linspace(0.1, 2.0, 200)         # Grid for price of good 2

# Allocate array to store equilibrium error for each price pair
E = np.empty((len(p1_grid), len(p2_grid)))

# Loop over all combinations of prices
for i, p1 in enumerate(p1_grid):
    for j, p2 in enumerate(p2_grid):
        # Compute Walras-based equilibrium error at (p1, p2)
        E[i, j] = equilibrium_error_walras(w, p1, p2, par)

# Find index of minimum equilibrium error
idx = np.unravel_index(np.argmin(E), E.shape)

# Extract equilibrium prices from grids
p1_ast = p1_grid[idx[0]]
p2_ast = p2_grid[idx[1]]

# Print equilibrium prices
print(f"Equilibrium prices:")
print(f"p1* = {p1_ast:.2f}")
print(f"p2* = {p2_ast:.2f}")

# Verify market clearing at equilibrium prices
sol_ast = market_clearing(w, p1_ast, p2_ast, par)

# Goods-market residuals (should be close to zero)
print("Good 1 residual:", sol_ast.res_good1)
print("Good 2 residual:", sol_ast.res_good2)

# Labor-market residual (zero by Walras' Law)
print("Labor residual (Walras check):", sol_ast.res_labor)

Assume the government care about the social welfare function:

$$
SWF = U - \kappa y_2^*
$$

Here $\kappa$ measures the social cost of carbon emitted by the production of $y_2$ in equilibrium.

**Question 3:** What values of $\tau$ and (implied) $T$ should the government choose to maximize $SWF$?

## 2. <a id='toc2_'></a>[Problem 2: Career choice model](#toc0_)

Consider a graduate $i$ making a choice between entering $J$ different career tracks. <br>
Entering career $j$ yields utility $u^k_{ij}$. This value is unknown to the graduate ex ante, but will ex post be: <br>
$$
    u_{i,j}^k = v_{j} + \epsilon_{i,j}^k
$$

They know that $\epsilon^k_{i,j}\sim \mathcal{N}(0,\sigma^2)$, but they do not observe $\epsilon^k_{i,j}$ before making their career choice. <br>

Consider the concrete case of $J=3$ with:
$$
\begin{align*}
    v_{1} &= 1 \\
    v_{2} &= 2 \\
    v_{3} &= 3
\end{align*}
$$

If the graduates know the values of $v_j$ and the distribution of $\epsilon_{i,j}^k$, they can calculate the expected utility of each career track using simulation: <br>
$$
    \mathbb{E}\left[ u^k_{i,j}\vert v_j \right] \approx v_j + \frac{1}{K}\sum_{k=1}^K \epsilon_{i,j}^k
$$

**Question 1:** Simulate and calculate expected utility and the average realised utility for $K=10000$ draws, for each career choice $j$.


In [9]:
# =========================
# Parameters
# =========================

par = SimpleNamespace()                      # Container for all model parameters

par.J = 3                                   # Number of career choices
par.N = 10                                  # Number of graduates (not used here)
par.K = 10_000                              # Number of simulation draws

par.F = np.arange(1, par.N + 1)             # Index set for graduates (1,...,N)
par.sigma = 2.0                             # Standard deviation of taste shocks

par.v = np.array([1.0, 2.0, 3.0])            # Deterministic utility components v_j
par.c = 1                                   # Placeholder parameter (not used here)


# =========================
# Simulation
# =========================

# Draw random utility shocks epsilon_{i,j}^k
# Shape: (K, J), one draw for each simulation k and career j
eps = np.random.normal(
    loc=0.0,                                # Mean of shocks
    scale=par.sigma,                        # Standard deviation of shocks
    size=(par.K, par.J)                     # Number of draws and careers
)

# Compute realised utilities u_{i,j}^k = v_j + epsilon_{i,j}^k
# Broadcasting adds v_j to each draw k
u = par.v + eps


# =========================
# Expected utility (simulation-based)
# =========================

# Approximate expected utility using simulation:
# E[u_{i,j} | v_j] ≈ v_j + (1/K) * sum_k epsilon_{i,j}^k
expected_utility = par.v + eps.mean(axis=0)


# =========================
# Average realised utility
# =========================

# Compute the average realised utility across simulations for each career
average_realised_utility = u.mean(axis=0)


# =========================
# Output
# =========================

print("Career choice results (K = 10000)")                                      # Header
print("-" * 40)                                                                 # Separator

# Loop over career choices
for j in range(par.J):
    print(f"Career {j+1}:")                                                     # Career index (1-based)
    print(f"  v_j                     = {par.v[j]:.2f}")                        # Deterministic utility
    print(f"  Expected utility        = {expected_utility[j]:.4f}")             # Simulated expectation
    print(f"  Average realised utility= {average_realised_utility[j]:.4f}")     # Realised mean
    print()                                                                     # Blank line for readability

Career choice results (K = 10000)
----------------------------------------
Career 1:
  v_j                     = 1.00
  Expected utility        = 1.0257
  Average realised utility= 1.0257

Career 2:
  v_j                     = 2.00
  Expected utility        = 2.0145
  Average realised utility= 2.0145

Career 3:
  v_j                     = 3.00
  Expected utility        = 3.0268
  Average realised utility= 3.0268



Now consider a new scenario: Imagine that the graduate does not know $v_j$. The *only* prior information they have on the value of each job, comes from their $F_{i}$ friends that work in each career $j$. After talking with them, they know the average utility of their friends (which includes their friends' noise term), giving them the prior expecation: <br>
$$
\tilde{u}^k_{i,j}\left( F_{i}\right) = \frac{1}{F_{i}}\sum_{f=1}^{F_{i}} \left(v_{j} + \epsilon^k_{f,j}\right), \; \epsilon^k_{f,j}\sim \mathcal{N}(0,\sigma^2)
$$
For ease of notation consider that each graduate have $F_{i}=i$ friends in each career. <br>

For $K$ times do the following: <br>
1. For each person $i$ draw $J\cdot F_i$ values of $\epsilon_{f,j}^{k}$, and calculate the prior expected utility of each career track, $\tilde{u}^k_{i,j}\left( F_{i}\right)$. <br>
Also draw their own $J$ noise terms, $\epsilon_{i,j}^k$
1. Each person $i$ chooses the career track with the highest expected utility: $$j_i^{k*}= \arg\max_{j\in{1,2\dots,J}}\left\{ \tilde{u}^k_{i,j}\left( F_{i}\right)\right\} $$
1. Store the chosen careers: $j_i^{k*}$, the prior expectation of the value of their chosen career: $\tilde{u}^k_{i,j=j_i^{k*}}\left( F_{i}\right)$, and the realized value of their chosen career track: $u^k_{i,j=j_i^{k*}}=v_{j=j_i^{k*}}+\epsilon_{i,j=j_i^{k*}}^k$.

Chosen values will be: <br>
$i\in\left\{1,2\dots,N\right\}, N=10$ <br>
$F_i = i$<br>
So there are 10 graduates. The first has 1 friend in each career, the second has 2 friends, ... the tenth has 10 friends.

**Question 2:** Simulate and visualize: For each type of graduate, $i$, the share of graduates choosing each career, the average subjective expected utility of the graduates, and the average ex post realized utility given their choice. <br>
That is, calculate and visualize: <br>
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \mathbb{I}\left\{ j=j_i^{k*} \right\}  \;\forall j\in\left\{1,2,\dots,J\right\}
\end{align*}
$$
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \tilde{u}^k_{ij=j_i^{k*}}\left( F_{i}\right)
\end{align*}
$$
And 
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} u^k_{ij=j_i^{k*}} 
\end{align*}
$$
For each graduate $i$.

In [ ]:
# Write your answer here 

After a year of working in their career, the graduates learn $u^k_{ij}$ for their chosen job $j_i^{k*}$ perfectly. <br>
The can switch to one of the two remaining careers, for which they have the same prior as before, but it will now include a switching cost of $c$ which is known.
Their new priors can be written as: 
$$
\tilde{u}^{k,2}_{ij}\left( F_{i}\right) = \begin{cases}
            \tilde{u}^k_{ij}\left( F_{i}\right)-c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

We will set $c=1$.

Their realized utility will be: <br>
$$
u^{k,2}_{ij}= \begin{cases}
            u_{ij}^k -c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

**Question 3:** Following the same approach as in question 2, find the new optimal career choice for each $i$, $k$. Then for each $i$, calculate the average subjective expected utility from their new optimal career choice, and the ex post realized utility of that career. Also, for each $i$, calculate the share of graduates that chooses to switch careers, conditional on which career they chose in the first year. <br>

In [ ]:
# write your answer here

## 3. <a id='toc3_'></a>[Problem 3: Barycentric interpolation](#toc0_)

**Problem:** We have a set of random points in the unit square,

$$
\mathcal{X} = \{(x_1,x_2)\,|\,x_1\sim\mathcal{U}(0,1),x_2\sim\mathcal{U}(0,1)\}.
$$

For these points, we know the value of some function $f(x_1,x_2)$,

$$
\mathcal{F} = \{f(x_1,x_2) \,|\, (x_1,x_2) \in \mathcal{X}\}.
$$

Now we want to approximate the value $f(y_1,y_2)$ for some  $y=(y_1,y_2)$, where $y_1\sim\mathcal{U}(0,1)$ and $y_2\sim\mathcal{U}(0,1)$.

**Building block I**

For an arbitrary triangle $ABC$ and a point $y$, define the so-called barycentric coordinates as:

$$
\begin{align*}
  r^{ABC}_1 &= \frac{(B_2-C_2)(y_1-C_1) + (C_1-B_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_2 &= \frac{(C_2-A_2)(y_1-C_1) + (A_1-C_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_3 &= 1 - r_1 - r_2.
\end{align*}
$$

If $r^{ABC}_1 \in [0,1]$, $r^{ABC}_2 \in [0,1]$, and $r^{ABC}_3 \in [0,1]$, then the point is inside the triangle.

We always have $y = r^{ABC}_1 A + r^{ABC}_2 B + r^{ABC}_3 C$.

**Building block II**

Define the following points:

$$
\begin{align*}
A&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}>y_{2}\\
B&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}<y_{2}\\
C&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}<y_{2}\\
D&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}>y_{2}.
\end{align*}
$$

**Algorithm:**

1. Compute $A$, $B$, $C$, and $D$. If not possible return `NaN`.
1. If $y$ is inside the triangle $ABC$ return $r^{ABC}_1 f(A) + r^{ABC}_2 f(B) + r^{ABC}_3 f(C)$.
1. If $y$ is inside the triangle $CDA$ return $r^{CDA}_1 f(C) + r^{CDA}_2 f(D) + r^{CDA}_3 f(A)$.
1. Return `NaN`.



**Sample:**

In [ ]:
rng = np.random.default_rng(2024)

X = rng.uniform(size=(50,2))
y = rng.uniform(size=(2,))


**Questions 1:** Find $A$, $B$, $C$ and $D$. Illustrate these together with $X$, $y$ and the triangles $ABC$ and $CDA$.

In [ ]:
# write your answer here

**Question 2:** Compute the barycentric coordinates of the point $y$ with respect to the triangles $ABC$ and $CDA$. Which triangle is $y$ located inside?

In [ ]:
# write your answer here

Now consider the function:
$$
f(x_1,x_2) = x_1 \cdot x_2
$$

In [ ]:
f = lambda x: x[0]*x[1]
F = np.array([f(x) for x in X])

**Question 3:** Compute the approximation of $f(y)$ using the full algorithm. Compare with the true value.

In [ ]:
# write your answer here

**Question 4:** Repeat question 3 for all points in the set $Y$.

In [ ]:
Y = [(0.2,0.2),(0.8,0.2),(0.8,0.8),(0.8,0.2),(0.5,0.5)]

In [ ]:
# write your answer here